# Use HYPER_PERSNL_Template Customized Notebook Template

In [2]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
infer_datetime_format=True

import numpy as np
import random

import matplotlib.pyplot as plt
plt.rcParams.update({'figure.figsize':(9,7), 'figure.dpi':120})

random.seed(0)

now1 = datetime.now()

In [3]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

In [4]:
table_name = 'CUSTOMER_ACCOUNT_DETAILS'

sf_df = my_session.sql("select * from {}".format(table_name))
account = sf_df.to_pandas()

In [6]:
table_name = 'CUSTOMER_TRANSACTION_DETAILS'

sf_df = my_session.sql("select * from {}".format(table_name))
transactions = sf_df.to_pandas()

In [7]:
transactions.columns

Index(['ACC_BBAN', 'ACC_IBAN', 'ACC_OPEN_DT', 'BALANCE', 'CUST_ID', 'EXPENSE',
       'EXPENSE_CATEGORY', 'BROWSER', 'DEVICE', 'DR_CR', 'LOGIN_TIME',
       'LOGOUT_TIME', 'TXN_DT', 'TXN_ID', 'TXN_AMOUNT', 'TXN_CODE',
       'TXN_REMARKS', 'TXN_TIME', 'TXN_TYPE'],
      dtype='object')

In [8]:
transactions.sort_values(by=['CUST_ID', 'ACC_BBAN', 'TXN_DT', 'TXN_TIME'], inplace=True) 

In [9]:
table_name = 'CUSTOMER_FINANCE_BREAKDOWN'

sf_df = my_session.sql("select * from {}".format(table_name))
financials = sf_df.to_pandas()

In [10]:
table_name = 'CUSTOMER_SUBSCRIPTION_DETAILS'

sf_df = my_session.sql("select * from {}".format(table_name))
subscriptions = sf_df.to_pandas()
subscriptions_orig = subscriptions.copy(deep=True)

In [11]:
table_name = 'CUSTOMER_TERM_DEPOSIT_DETAILS'

sf_df = my_session.sql("select * from {}".format(table_name))
term_deposit = sf_df.to_pandas()

In [12]:
table_name = 'CUSTOMER_LOAN_DETAILS'

sf_df = my_session.sql("select * from {}".format(table_name))
loan = sf_df.to_pandas()

In [13]:
table_name = 'CUSTOMER_CREDITCARD_DETAILS'

sf_df = my_session.sql("select * from {}".format(table_name))
credit_card = sf_df.to_pandas()

In [15]:
x1 = list(set(transactions['CUST_ID']))

In [19]:
account

,CUST_ID,ACC_IBAN,ACC_BBAN,ACC_OPEN_DT,BALANCE,ACC_STATUS,ACC_TYPE
0,4185,GB90EMIC89482763544524,GDJY66571183273954,2001-03-22,$50.57,Active,Credit
1,291,GB83VYSJ52094207716710,OVWN34297715684777,2010-08-29,"$74,111.13",Closed,Current
2,7497,GB17OCSO08226852523762,DLFO12501444293678,2000-09-04,$669.14,Active,Loan
3,7497,GB26RRMJ41734703010562,SZPN32742192244415,2010-09-05,$0.03,Active,Credit
4,7497,GB13JBDZ14742645466984,DAGC95804976124339,2006-02-14,$0.71,Closed,Credit
...,...,...,...,...,...,...,...
3101,2137,GB79MEEM68164816755595,JGQZ63653199537639,2020-07-08,$191.20,Active,Savings
3102,2137,GB72FWKV94464237567600,LCGK26173403352193,2011-06-05,"$54,340.79",Closed,Savings
3103,3899,GB97XFCA41641435894700,IQWX43085486211593,2006-01-08,$906.56,Active,Loan
3104,3899,GB72HHTA49364267690196,PFVP16819625415314,2014-10-14,$17.61,Closed,Loan


In [22]:
#bank_name = list(set(account['BANK_NAME'])) --> Unable to find Bank_Name in account table.
bank_name = ['UBS',
'Bank of America',
'Discover Financial',
'Credit Suisse',
'Santander Bank',
'American Express',
'HSBC Bank USA',
'JPMorgan Chase',
'BankUnited',
'Citigroup',
'Barclays',
'RBC Bank',
'Citizens Financial Group',
'Capital One',
'Goldman Sachs',
'Wells Fargo',
'Morgan Stanley']

# Reversing a list using slicing technique

In [21]:
def Reverse(lst):
    new_lst = lst[::-1]
    return new_lst

# Initiate the DataFrames 

In [23]:
EOD_Acc_Level = pd.DataFrame(columns = ['CUST_ID', 'ACC_IBAN', 'ACC_BBAN', 'DT', 'EOD_BALANCE', 
                                  'EOD_TOTAL_CR', 'EOD_TOTAL_DR', 'EOD_TOTAL_INCOME', 
                                  'EOD_TOTAL_EXPENSE'])

In [24]:
EOD_Cust_Level = pd.DataFrame(columns = ['CUST_ID', 'DT', 'EOD_BALANCE', 'EOD_TOTAL_CR', 'EOD_TOTAL_DR', 
                                  'EOD_TOTAL_INCOME', 'EOD_TOTAL_BALANCE'])

In [25]:
EOW_Acc_Level = pd.DataFrame(columns = ['CUST_ID', 'ACC_IBAN', 'ACC_BBAN', 'WK_NUM-YR', 'WK_START_DT', 'Wk_END_DT', 'EOW_BALANCE', 'EOW_TOTAL_DR', 
                                        'EOW_TOTAL_CR','EOW_TOTAL_INCOME', 'EOW_TOTAL_EXPENSE', 'SLOPE_BALANCE',
                              'SLOPE_TOTAL_CR', 'SLOPE_TOTAL_DR', 'SLOPE_TOTAL_INCOME','SLOPE_TOTAL_EXPENSE'])

In [26]:
EOW_Cust_Level = pd.DataFrame(columns = ['CUST_ID', 'WK_NUM-YR', 'WK_START_DT', 'Wk_END_DT', 'EOW_BALANCE',
       'EOW_TOTAL_CR', 'EOW_TOTAL_DR', 'EOW_TOTAL_INCOME', 'EOW_TOTAL_EXPENSE','SLOPE_BALANCE', 
       'SLOPE_TOTAL_CR', 'SLOPE_TOTAL_DR', 'SLOPE_TOTAL_INCOME','SLOPE_TOTAL_EXPENSE'])

In [27]:
EOM_Acc_Level = pd.DataFrame(columns = ['CUST_ID', 'ACC_IBAN', 'ACC_BBAN', 'MTH_NUM-YR', 'EOM_BALANCE',
       'EOM_TOTAL_DR', 'EOM_TOTAL_CR', 'EOM_TOTAL_INCOME', 'EOM_TOTAL_EXPENSE', 'SLOPE_BALANCE', 
       'SLOPE_TOTAL_CR', 'SLOPE_TOTAL_DR',  'SLOPE_TOTAL_INCOME','SLOPE_TOTAL_EXPENSE'])

In [28]:
EOM_Cust_Level = pd.DataFrame(columns = ['CUST_ID', 'MTH_NUM-YR', 'EOM_BALANCE', 'EOM_TOTAL_CR', 
       'EOM_TOTAL_DR', 'EOM_TOTAL_INCOME', 'EOM_TOTAL_EXPENSE', 'SLOPE_BALANCE',
       'SLOPE_TOTAL_CR', 'SLOPE_TOTAL_DR',  'SLOPE_TOTAL_INCOME','SLOPE_TOTAL_EXPENSE'])

In [29]:
Cust_Dashboard = pd.DataFrame(columns = ['CUST_ID', 'BALANCE', 'AVG_BALANCE', 'AVG_INCOME', 'AVG_EXPENSE', 'BANK_NAME'])   

# Aggregation logic for Daily, Weekly and Monthly 

In [116]:
transactions['TXN_DT'] = pd.to_datetime(transactions['TXN_DT'], format='%m/%d/%Y')

In [146]:
for i in x1: ### This is coming from account table
    ibans = list(set(transactions[transactions['CUST_ID']==i]['ACC_IBAN']))
    for j in ibans:  ### for a given customer pickup one IBAN at a time
        txn_temp = transactions.loc[(transactions['CUST_ID']==i) & (transactions['ACC_IBAN']==j), ['ACC_BBAN', 'ACC_IBAN','BALANCE', 'TXN_DT', 'DR_CR', 'TXN_AMOUNT', 'TXN_CODE']]    ### This is at account level
        txn_Dr = txn_temp.loc[(txn_temp['DR_CR']=='Dr')]    ### This is at account level
        txn_Cr = txn_temp.loc[(txn_temp['DR_CR']=='Cr')]    ### This is at account level
        ##txn_temp['txn_Dt'] = txn_temp['txn_Dt'].astype(str)

        #### Process as per balance column for EOD Balance ##############
        #Dts = txn_temp['TXN_DT'].drop_duplicates() 
        Dts = pd.to_datetime(txn_temp['TXN_DT'].drop_duplicates(), format='%m/%d/%Y')
        
        sdate = datetime.strptime(str(min(Dts))[:10], '%Y-%m-%d')
        edate = datetime.strptime(str(max(Dts))[:10], '%Y-%m-%d')        
        
        dt_range = pd.date_range(sdate,edate-timedelta(days=1),freq='d')
        hn = len(dt_range)
        eod_balance = []

        for di in Dts:
            eod_balance = eod_balance + [txn_temp.loc[txn_temp['TXN_DT']==di, ['TXN_DT', 'BALANCE']].iloc[-1, 1]]
        temp_Acc_Daily = pd.DataFrame({'DT': list(Dts), 'EOD_BALANCE': eod_balance})
        temp_Acc_Daily.reset_index(inplace=True) 
        temp_Acc_Daily['DT']=temp_Acc_Daily['DT'].astype('str')
        delta_Acc_Daily = pd.DataFrame({'CUST_ID': [i]*hn, 'ACC_IBAN': [j]*hn, 'ACC_BBAN':[txn_temp.iloc[0,0]]*hn, 
                                        'DT': dt_range})
        delta_Acc_Daily['DT']=delta_Acc_Daily['DT'].astype('str')
        delta_Acc_Daily = pd.merge(delta_Acc_Daily, temp_Acc_Daily, on='DT', how='left')
        
        ## delta_Acc_Daily.to_csv('delta_Acc_Daily.csv', index=False)
        delta_Acc_Daily.drop('index', axis=1, inplace=True)
        gt0_values = delta_Acc_Daily.loc[delta_Acc_Daily['EOD_BALANCE']>0, 'EOD_BALANCE']
        gt0_idx = list(gt0_values.index) + [len(delta_Acc_Daily)]
        eod_balance=[]
        for idx in range(len(gt0_idx)-1):
            eod_balance = eod_balance+[gt0_values[gt0_idx[idx]]]*(gt0_idx[idx+1]-gt0_idx[idx])
        delta_Acc_Daily['EOD_BALANCE'] = eod_balance

        #### Process Debit amount as per dr_cr column ##############
        Dr_sum = pd.DataFrame(txn_Dr.groupby('TXN_DT')['TXN_AMOUNT'].sum())
        Dr_sum['TXN_DT'] = Dr_sum.index
        Dr_sum.index = range(len(Dr_sum))
        Dr_sum.rename(columns = {'TXN_DT':'DT', 'TXN_AMOUNT':'EOD_TOTAL_DR'}, inplace=True)

        #### Process Credit amount as per dr_cr column ##############
        Cr_sum = pd.DataFrame(txn_Cr.groupby('TXN_DT')['TXN_AMOUNT'].sum())
        Cr_sum['TXN_DT'] = Cr_sum.index
        Cr_sum.index = range(len(Cr_sum))
        Cr_sum.rename(columns = {'TXN_DT':'DT', 'TXN_AMOUNT':'EOD_TOTAL_CR'}, inplace=True)        
        
        ### Now merge the dataframes ###########
        Dr_sum['DT']=Dr_sum['DT'].astype('str')
        Cr_sum['DT']=Cr_sum['DT'].astype('str')
        
        delta_Acc_Daily = pd.merge(delta_Acc_Daily, Dr_sum, on='DT', how='left')
        delta_Acc_Daily = pd.merge(delta_Acc_Daily, Cr_sum, on='DT', how='left')
        
        delta_Acc_Daily['EOD_TOTAL_INCOME']=delta_Acc_Daily['EOD_TOTAL_CR']
        delta_Acc_Daily['EOD_TOTAL_EXPENSE']=delta_Acc_Daily['EOD_TOTAL_DR']               
        ##print(delta_Acc_Daily.shape)
        delta_Acc_Daily.fillna(0, inplace=True)

        #EOD_Acc_Level=EOD_Acc_Level.append(delta_Acc_Daily) --> Throwing error
        EOD_Acc_Level = pd.concat([EOD_Acc_Level, delta_Acc_Daily], ignore_index=True)
        

        #### Process for EOW Balance ##############        
        temp_Acc_Wkly = delta_Acc_Daily
        temp_Acc_Wkly['WK_NUM-YR'] = [datetime.strptime(x, '%Y-%m-%d').strftime('%Y-W%W') for x in temp_Acc_Wkly['DT']]
        temp_Acc_Wkly.replace(np.nan, 0, inplace=True)
        temp_Dt = [datetime.strptime(x, '%Y-%m-%d') for x in temp_Acc_Wkly['DT']]
        temp_Acc_Wkly['WK_START_DT'] = [x - timedelta(days=x.weekday()) for x in temp_Dt]
        temp_Acc_Wkly['WK_END_DT'] = [x + timedelta(days=6) for x in temp_Acc_Wkly['WK_START_DT']]

        ##transactions.loc[(transactions['Cust_ID']==i) & (transactions['Acc_IBAN']==j), ['Acc_BBAN', 'Balance', 'txn_Dt', 'dr_cr', 'txn_amount', 'txn_code']]    ### This is at account level
        
        ##temp = temp_Acc_Wkly.groupby('Wk_Num-Yr')[['EOD_Total_Dr', 'EOD_Total_Cr', 'EOD_Total_Income', 'EOD_Total_Expense']].sum()
        
        ##delta_Acc_Wkly = pd.DataFrame(temp_Acc_Wkly.groupby('Wk_Num-Yr').agg({'EOD_Balance':'last', 'EOD_Total_Dr':'sum', 'EOD_Total_Cr':'sum', 'EOD_Total_Income':'sum', 'EOD_Total_Expense':'sum'}))
        
        delta_Acc_Wkly = pd.DataFrame(temp_Acc_Wkly.groupby(['CUST_ID', 'ACC_IBAN', 'ACC_BBAN','WK_NUM-YR', 'WK_START_DT', 'WK_END_DT']).agg({'EOD_BALANCE':'last', 'EOD_TOTAL_DR':'sum', 'EOD_TOTAL_CR':'sum', 'EOD_TOTAL_INCOME':'sum', 'EOD_TOTAL_EXPENSE':'sum'}))
        delta_Acc_Wkly.reset_index(inplace=True)
                
        delta_Acc_Wkly.replace(np.nan, 0, inplace=True)
        ##temp_Dt = [datetime.strptime(x, '%Y-%m-%d') for x in temp_Acc_Wkly['Dt']]
        ##delta_Acc_Wkly['Wk_Start_Dt'] = set([x - timedelta(days=x.weekday()) for x in temp_Dt])
        ##delta_Acc_Wkly['Wk_End_Dt'] = set([x + timedelta(days=6) for x in delta_Acc_Wkly['Wk_Start_Dt']])

        #### Find the slope for accounts at daily level for each block of Wk_Num-Yr ######
        slope_cols = ['SLOPE_BALANCE', 'SLOPE_TOTAL_CR', 'SLOPE_TOTAL_DR', 'SLOPE_TOTAL_INCOME', 'SLOPE_TOTAL_EXPENSE']        
        amount_cols = ['EOD_BALANCE', 'EOD_TOTAL_DR', 'EOD_TOTAL_CR', 'EOD_TOTAL_INCOME', 'EOD_TOTAL_EXPENSE']

        for iamt in range(len(amount_cols)):
            slope_temp = []
            for iwk in sorted(list(set(temp_Acc_Wkly['WK_NUM-YR']))):
                slope_EOD_Bal = list(temp_Acc_Wkly[amount_cols[iamt]].loc[(temp_Acc_Wkly['CUST_ID']==i) & (temp_Acc_Wkly['ACC_IBAN']==j) & (temp_Acc_Wkly['WK_NUM-YR']==iwk)])
                #if any([len(slope_EOD_Bal)<2]+[math.isnan(x) for x in slope_EOD_Bal]):
                if len(slope_EOD_Bal)<2:
                    slope_temp = slope_temp + [0]
                else:
                    slope_temp = slope_temp + [round(np.polyfit(list(range(len(slope_EOD_Bal))), slope_EOD_Bal, 1)[0], 2)]
            delta_Acc_Wkly[slope_cols[iamt]]=slope_temp

        
        ##r = datetime.datetime.strptime(temp_Acc_Wkly['Wk_Num-Yr'][0] + '-1', '%G-W%V-%u')
        delta_Acc_Wkly.rename(columns = {'EOD_BALANCE':'EOW_BALANCE', 'EOD_TOTAL_CR':'EOW_TOTAL_DR', 'EOD_TOTAL_DR':'EOW_TOTAL_CR', 'EOD_TOTAL_INCOME':'EOW_TOTAL_INCOME', 'EOD_TOTAL_EXPENSE':'EOW_TOTAL_EXPENSE'}, inplace=True)
        delta_Acc_Wkly.fillna(0, inplace=True)

        #EOW_Acc_Level = EOW_Acc_Level.append(delta_Acc_Wkly) -> Throwing Error
        EOW_Acc_Level = pd.concat([EOW_Acc_Level, delta_Acc_Wkly], ignore_index=True)

        #### Process for EOM Balance ##############
        temp_Acc_Mthly = delta_Acc_Daily
        temp_Dt = [datetime.strptime(x, '%Y-%m-%d') for x in temp_Acc_Mthly['DT']]
        temp_Acc_Mthly['MTH_NUM-YR'] = [x.strftime('%Y-%m') for x in temp_Dt]
        delta_Acc_Mthly = pd.DataFrame(temp_Acc_Mthly.groupby(['CUST_ID', 'ACC_IBAN', 'ACC_BBAN','MTH_NUM-YR']).agg({'EOD_BALANCE':'last', 'EOD_TOTAL_DR':'sum', 'EOD_TOTAL_CR':'sum', 'EOD_TOTAL_INCOME':'sum', 'EOD_TOTAL_EXPENSE':'sum'}))
        delta_Acc_Mthly.reset_index(inplace=True)
        delta_Acc_Mthly.fillna(0, inplace=True)        
        delta_Acc_Mthly.replace(np.nan, 0, inplace=True)

        #### Find the slope for accounts at weekly level for each block of Mth_Num-Yr ######
        slope_cols = ['SLOPE_BALANCE', 'SLOPE_TOTAL_CR', 'SLOPE_TOTAL_DR', 'SLOPE_TOTAL_INCOME', 'SLOPE_TOTAL_EXPENSE']     
        amount_cols = ['EOD_BALANCE', 'EOD_TOTAL_DR', 'EOD_TOTAL_CR', 'EOD_TOTAL_INCOME', 'EOD_TOTAL_EXPENSE']

        for iamt in range(len(amount_cols)):
            slope_temp = []
            for imth in sorted(set(temp_Acc_Mthly['MTH_NUM-YR'])):
                slope_EOD_Bal = list(temp_Acc_Mthly[amount_cols[iamt]].loc[(temp_Acc_Mthly['CUST_ID']==i) & (temp_Acc_Mthly['ACC_IBAN']==j) & (temp_Acc_Mthly['MTH_NUM-YR']==imth)])
                #if any([len(slope_EOD_Bal)<2]+[math.isnan(x) for x in slope_EOD_Bal]):
                if len(slope_EOD_Bal)<2:
                    slope_temp = slope_temp + [0]
                else:
                    slope_temp = slope_temp + [round(np.polyfit(list(range(len(slope_EOD_Bal))), slope_EOD_Bal, 1)[0], 2)]
            delta_Acc_Mthly[slope_cols[iamt]]=slope_temp
        
        delta_Acc_Mthly.rename(columns = {'EOD_BALANCE':'EOM_BALANCE', 'EOD_TOTAL_DR':'EOM_TOTAL_DR', 'EOD_TOTAL_CR':'EOM_TOTAL_CR', 'EOD_TOTAL_INCOME':'EOM_TOTAL_INCOME', 'EOD_TOTAL_EXPENSE':'EOM_TOTAL_EXPENSE'}, inplace=True)
        
        #EOM_Acc_Level = EOM_Acc_Level.append(delta_Acc_Mthly) --> Throwing error
        EOM_Acc_Level = pd.concat([EOM_Acc_Level, delta_Acc_Mthly], ignore_index=True)

print('Account level tables generated - All')

Account level tables generated - All


# Generate Customer Level Tables - All

In [147]:
EOD_Cust_Level = pd.DataFrame(EOD_Acc_Level.groupby(['CUST_ID', 'DT'])[['EOD_BALANCE', 'EOD_TOTAL_CR', 'EOD_TOTAL_DR', 'EOD_TOTAL_INCOME','EOD_TOTAL_EXPENSE']].sum())
EOD_Cust_Level.reset_index(inplace=True)
EOD_Cust_Level.fillna(0, inplace=True)
EOD_Cust_Level.replace(np.nan, 0, inplace=True)
EOD_Cust_Level['BANK_NAME'] = 'All'

In [148]:
EOW_Cust_Level = pd.DataFrame(EOW_Acc_Level.groupby(['CUST_ID', 'WK_NUM-YR','WK_START_DT', 'WK_END_DT'])[['EOW_BALANCE', 'EOW_TOTAL_CR', 'EOW_TOTAL_DR', 'EOW_TOTAL_INCOME','EOW_TOTAL_EXPENSE']].sum())
EOW_Cust_Level.reset_index(inplace=True)
EOW_Cust_Level.fillna(0, inplace=True)
EOW_Cust_Level.replace(np.nan, 0, inplace=True)
EOW_Cust_Level['BANK_NAME'] = 'All'

In [150]:
#### Find the slope for Customers at daily level for each block of Wk_Num-Yr ######
slope_cols = ['SLOPE_BALANCE', 'SLOPE_TOTAL_CR', 'SLOPE_TOTAL_DR', 'SLOPE_TOTAL_INCOME', 'SLOPE_TOTAL_EXPENSE']        
amount_cols = ['EOD_BALANCE', 'EOD_TOTAL_DR', 'EOD_TOTAL_CR', 'EOD_TOTAL_INCOME', 'EOD_TOTAL_EXPENSE']

temp_Cust_Daily = EOD_Cust_Level
temp_Cust_Daily['WK_NUM-YR'] = [datetime.strptime(x, '%Y-%m-%d').strftime('%Y-W%W') for x in temp_Cust_Daily['DT']]

for iamt in range(len(amount_cols)):
    slope_temp = []
    for i in x1:
        temp_Wk_Num = sorted(set(temp_Cust_Daily['WK_NUM-YR'].loc[temp_Cust_Daily['CUST_ID']==i]))
        for iwk in temp_Wk_Num:
            temp_Daily = temp_Cust_Daily[amount_cols[iamt]].loc[(temp_Cust_Daily['CUST_ID']==i) & (temp_Cust_Daily['WK_NUM-YR']==iwk)]
            ##print(temp_Daily.shape)
            slope_EOD = list(temp_Cust_Daily[amount_cols[iamt]].loc[(temp_Cust_Daily['CUST_ID']==i) & (temp_Cust_Daily['WK_NUM-YR']==iwk)])
            #if any([len(slope_EOD)<2]+[math.isnan(x) for x in slope_EOD]):
            if len(slope_EOD)<2:
                slope_temp = slope_temp + [0]
            else:
                slope_temp = slope_temp + [round(np.polyfit(list(range(len(slope_EOD))), slope_EOD, 1)[0], 2)]
    EOW_Cust_Level[slope_cols[iamt]]=slope_temp

In [151]:
EOM_Cust_Level = pd.DataFrame(EOM_Acc_Level.groupby(['CUST_ID', 'MTH_NUM-YR'])[['EOM_BALANCE', 'EOM_TOTAL_CR', 'EOM_TOTAL_DR', 'EOM_TOTAL_INCOME','EOM_TOTAL_EXPENSE']].sum())
EOM_Cust_Level.reset_index(inplace=True)
EOM_Cust_Level.fillna(0, inplace=True)
EOM_Cust_Level.replace(np.nan, 0, inplace=True)
EOM_Cust_Level['BANK_NAME'] = 'All'
EOM_Cust_Level['YR'] = [x[:4] for x in EOM_Cust_Level['MTH_NUM-YR']]
EOM_Cust_Level['MTH_NUM'] = [x[-2:] for x in EOM_Cust_Level['MTH_NUM-YR']]

Mth_Name = {'01':'January', '02':'February', '03':'March', '04':'April', '05':'May', '06':'June', '07':'July', '08':'August', '09':'September', '10':'October', '11':'November', '12':'December'}

EOM_Cust_Level['MTH'] = [Mth_Name[x] for x in EOM_Cust_Level['MTH_NUM']]
##EOM_Cust_Level.drop('Mth_Num', axis=1, inplace=True)

In [153]:
#### Find the slope for Customers at weekly level for each block of Mth_Num-Yr ######

temp_Cust_Daily = EOD_Cust_Level
temp_Cust_Daily['MTH_NUM-YR'] = [datetime.strptime(x, '%Y-%m-%d').strftime('%Y-%m') for x in temp_Cust_Daily['DT']]
    
for iamt in range(len(amount_cols)):
    slope_temp = []
    for i in x1:
        temp_Mth_Num = sorted(set(temp_Cust_Daily['MTH_NUM-YR'].loc[temp_Cust_Daily['CUST_ID']==i]))
        for imth in temp_Mth_Num:
            temp_Daily = temp_Cust_Daily[amount_cols[iamt]].loc[(temp_Cust_Daily['CUST_ID']==i) & (temp_Cust_Daily['WK_NUM-YR']==imth)]
            ##print(temp_Daily.shape)
            slope_EOD = list(temp_Cust_Daily[amount_cols[iamt]].loc[(temp_Cust_Daily['CUST_ID']==i) & (temp_Cust_Daily['MTH_NUM-YR']==imth)])
            #if any([len(slope_EOD)<2]+[math.isnan(x) for x in slope_EOD]):
            if len(slope_EOD)<2:
                slope_temp = slope_temp + [0]
            else:
                slope_temp = slope_temp + [round(np.polyfit(range(len(slope_EOD)), slope_EOD, 1)[0], 2)]
    EOM_Cust_Level[slope_cols[iamt]]=slope_temp

####### Clean-up: Drop / Add columns etc ##############

EOD_Cust_Level.drop(['WK_NUM-YR', 'MTH_NUM-YR'], axis=1, inplace=True)

print('Customer level tables generated - All')

Customer level tables generated - All


# Generation of Customer Dashboard Table

In [156]:
for i in x1:
    Cust_Daily = EOD_Cust_Level.loc[EOD_Cust_Level['CUST_ID']==i, 'EOD_BALANCE']    ### This is at account level
    balance = list(Cust_Daily)[-1] if len(Cust_Daily)>0 else 0
    Cust_Mthly = EOM_Cust_Level.loc[EOM_Cust_Level['CUST_ID']==i, ['EOM_BALANCE', 'EOM_TOTAL_INCOME', 'EOM_TOTAL_EXPENSE']]
    if len(Cust_Mthly)>0:
        n1 = min(6, len(Cust_Mthly))
        Avg_Balance = round(np.mean(list(Cust_Mthly['EOM_BALANCE'])[-n1:]), 2)
        Avg_Income = round(np.mean(list(Cust_Mthly['EOM_TOTAL_INCOME'])[-n1:]), 2)
        Avg_Expense = round(np.mean(list(Cust_Mthly['EOM_TOTAL_EXPENSE'])[-n1:]), 2)
    else:
        Avg_Balance, Avg_Income, Avg_Expense = 0, 0, 0
    
    delta_Cust_Dashboard = pd.DataFrame({'CUST_ID':[i], 'BALANCE':[balance], 
                                         'AVG_BALANCE':[Avg_Balance], 
                                         'AVG_INCOME':[Avg_Income], 'AVG_EXPENSE':[Avg_Expense]})
    
            
    #Cust_Dashboard = Cust_Dashboard.append(delta_Cust_Dashboard) --> Throwing Error
    Cust_Dashboard = pd.concat([Cust_Dashboard,delta_Cust_Dashboard], ignore_index=True)

# Handled Error in Credit Card csv (Annual Fee missing)

In [161]:
Cust_Dashboard.reset_index(inplace=True)
Cust_Dashboard.drop('index', axis=1, inplace=True)

Fees_Charges = pd.DataFrame(subscriptions.groupby(['CUST_ID'])[['AMOUNT']].sum())
Fees_Charges.reset_index(inplace=True)

#Fees_CC = credit_card.loc[credit_card['ANNUAL_FEE']>0, ['CUST_ID', 'ANNUAL_FEE']]
#Fees_CC = pd.DataFrame(Fees_CC.groupby('CUST_ID')['ANNUAL_FEE'].sum())
#Fees_CC.reset_index(inplace=True)
#Fees_Charges = pd.merge(Fees_Charges, Fees_CC, how='outer', on = 'CUST_ID')
#Fees_Charges['FEES_CHARGES'] = Fees_Charges['AMOUNT']+Fees_Charges['ANNUAL_FEE']
Fees_Charges['FEES_CHARGES'] = Fees_Charges['AMOUNT']

Cust_Dashboard = pd.merge(Cust_Dashboard, Fees_Charges[['CUST_ID', 'FEES_CHARGES']], on = 'CUST_ID', how = 'left')

Cust_Dashboard = pd.merge(Cust_Dashboard, financials, on = 'CUST_ID', how = 'left')

##Cust_Dashboard['Peer_Credit_Score']=[int(round(np.mean([random.choice(list(financials['Credit_Score'])) for _ in range(10)]), 0)) for _ in range(len(x1))]

In [162]:
similar_subsc = pd.DataFrame(subscriptions.groupby(['CUST_ID', 'CATEGORY']).size())
similar_subsc.reset_index(inplace=True)
similar_subsc.rename(columns={0:'SUBSCRIPTIONS_COUNT'}, inplace=True)

subscriptions = pd.merge(subscriptions, similar_subsc, how='left', left_on = ['CUST_ID', 'CATEGORY'], right_on = ['CUST_ID', 'CATEGORY'])
subscriptions['BANK_NAME'] = 'All'
subscriptions.sort_values(by = ['CUST_ID', 'CATEGORY'], ascending = [True, True], inplace=True)

# Upcoming Payouts - Present month due date

In [ ]:
Upcoming_TD = term_deposit.loc[(term_deposit['DEPOSIT_TYPE']=='Recurring Deposit') & (term_deposit['DUE_DT']>=now1.day), ['CUST_ID', 'DUE_DT', 'MTHLY_PAYMENT']]
Upcoming_TD['TABLE_NAME'] = 'Term_Deposit'
Upcoming_TD['TYPE'] = 'Term Deposit'
Upcoming_OP = Upcoming_TD

Upcoming_Loan = loan.loc[(loan['LOAN_STATUS'].isin(['Active', 'Default'])) & (loan['DUE_DT']>=now1.day), ['CUST_ID', 'DUE_DT', 'MTHLY_PAYMENT', 'Loan_Type']]
Upcoming_Loan['TABLE_NAME'] = 'Loan'
Upcoming_Loan.rename(columns={'LOAN_TYPE':'TYPE'}, inplace=True)
Upcoming_OP = Upcoming_OP.append(Upcoming_Loan)

Upcoming_CC = credit_card.loc[(credit_card['Status']=='Active') & (credit_card['DUE_DT']>=now1.day), ['CUST_ID', 'DUE_DT', 'MTHLY_PAYMENT', 'Card_Provider']]
Upcoming_CC['TABLE_NAME'] = 'Credit Card'
Upcoming_CC.rename(columns={'Card_Provider':'Type'}, inplace=True)
Upcoming_OP = Upcoming_OP.append(Upcoming_CC)

Upcoming_OP.sort_values(by = ['Cust_ID', 'Mthly_Payment', 'DUE_DT'], ascending = [True, False, True], inplace=True)
Upcoming_OP['DUE_DT_STR'] = [str(x)+now1.strftime(' %b')+str(now1.strftime('%y')) for x in Upcoming_OP['DUE_DT']]
